# Quick plots - CTSM - Monthly 
## Simple way to quickly look at monthly history files (*h0*).  

This tutorial is an introduction to [xarray](https://docs.xarray.dev/en/stable/user-guide/terminology.html) and [matplotlib](https://matplotlib.org/stable/index.html). There is plenty more information to be found at the documentation for these libraries.

This tutorial can be be run either on data from cases that you ran earlier, or can be run on pre-staged data.

In this tutorial you will find steps and instructions to:

1. Load python libraries needed for plotting
2. Locate monthly history files
3. Load datasets with xarray
4. Plot the data
------

# 1. Load Datasets

## 1.1 Load Python Libraries
We always start by loading in the libraries we're going to use for the script. There are more libraries being loaded here than we'll likely use, but this list is a good one to get started for most of your plotting needs.


In [ ]:
import os
import time
import datetime

import numpy as np
import pandas as pd
import xarray as xr

from glob import glob
from os.path import join

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from neon_utils import fix_time_h0

In [ ]:
# It's helpful to document the version of some tools that are quickly changing
print('xarray '+xr.__version__) # was working with 2023.5.0

## 1.2 Point to history files 

### 1.2.1 Where are my simulation results?
After your simulations finish, history files are all saved in your `/scratch/NEON_cases/archive/` directory

We can print the cases we have to look at using bash magic, `%%bash` or `!` which turns the python cell block below into a bash cell.  

In [ ]:
%%bash
ls ~/scratch/NEON_cases/archive/

<div class="alert alert-block alert-info">
<b>Note</b> you can accomplish the same thing with the following.

> `!ls ~/scratch/NEON_cases/archive/`
    
</div>

<div class="alert alert-block alert-info">
<b>Note</b> if you prefer to look at example data instead of your own data, you can read in data located at `/scratch/data/NEONv2/hist`. We'll go over this in the next section.

</div>

---

### 1.2.2 Point to the data folder with history files 
**We'll set the following:**
- site to look at; 
- path to our archive directory;
- directory with input data (where history files are found).
By doing this more generally, it makes the script easier to modify for different sites.

<div class="alert alert-block alert-info">
<b>Note</b> you can accomplish the same thing with the following.

> `!ls ~/scratch/NEON_cases/archive/`
    
</div>

---

### 1.2.2 Point to the data folder with history files 
**We'll set the following:**
- site to look at
- path to our archive directory
- directory with input data (where history files are found)
By doing this more generally, it makes the script easier to modify for different sites.

In [ ]:
neon_site = 'ABBY'  # NEON site we're going to look at
archive = '~/scratch/NEON_cases/archive' # Path to archive directory

# If you prefer to look at example data, you can uncomment the following line
# archive = '~/../../scratch/data/NEONv2/hist'

# This unpacks the and expands the shortcut, '~', we used above
archive = os.path.realpath(os.path.expanduser(archive)) 

# Create a path to the data folder
data_folder = archive+'/'+neon_site+'.transient/lnd/hist'

data_folder

**Is this path for input data, `data_folder`, correct?** 

*HINT:* You can check in the terminal window or use bash magic (`%%bash`) and then list the contents of `data_folder` with `ls` in the same cell.

---

### 1.2.3 Create some functions we'll use when opening the data
1. `preprocess` will limit the number of variables we're reading in. This is an xarray feature that helps save time (and memory resources).
2. `fix_time` corrects annoying features related to how CTSM history files handle time and is provided in `neon_utils.py`
*Don't worry too much about the details of these functions right now*.

In [ ]:
# -- read all variables from the netcdf files
#    this just drops an unused coordinate variable (lndgrid) from the dataset
def preprocess_all (ds):
    ds_new= ds.isel(lndgrid=0) 
    return ds_new

# -- read some of the variables from the netcdf files, 
#    this will make things faster, but requires you to list the variables you want to look at
def preprocess_some (ds):
    variables = ['H2OSOI', 'TSOI']
    ds_new= ds[variables].isel(lndgrid=0)
    return ds_new

Now we have created the functions needed to manipulate our datasets

---

### 1.2.4 List all the files we're going to open
The monthly history output (**'h0' files**) are written out for NEON cases. 

To open all of these files we're going to need to know their names.  This can be done if we:
- Create an empty list `[]` of simulation files that is
- `.extend`ed with a 
- `sorted` list of files generated with the 
- `glob` function in python of the 
- `*h0*`files in our `data_folder` 

You'll notice that **all of this gets combined in a single line of code** that runs through a 
- `for` loop over defined simulation years (written as a list of strings)

<div class="alert alert-block alert-info">
<b>Note</b> If you're new to python it's dense, but efficient.  I actually borrowed a bunch this code from a colleague, Negin Sobhani, who's good at python! Sharing code is really helpful. 
</div>



In [ ]:
# Create an empty list of all the file names to extend
sim_files = []

# If you want to choose a few particular years, you can use this loop
# years = [2021, 2022]
# for year in years:
#    sim_files.extend(sorted(glob(join(data_folder,"*h0."+year+"*.nc"))))

sim_files.extend(sorted(glob(join(data_folder,"*h0.*.nc"))))

How many files are you going have to read in?  What is the last day of the simulation you'll be looking at?

In [ ]:
print("Total number of simulation files: ", len(sim_files), "files")
print("Last simulation file:", sim_files[-1])

---

### 1.2.5 Read in the data
`xr.open_mfdataset` will open all of these data files and concatenate them into a single **xarray dataset**.

We are also going to use the `preprocess` and `fix_time` functions in this step.

In [ ]:
start = time.time()
print ("Reading in data for "+neon_site)

# Just reading *some* of the data here, could use preprocess_all instead.
ds_ctsm = xr.open_mfdataset(sim_files, decode_times=True, combine='by_coords',
                            preprocess=preprocess_some)
ds_ctsm = fix_time_h0(ds_ctsm)

end = time.time()
print("Reading all simulation files took:", end-start, "s.")


### Print the dataset you're working with ds_ctsm

In [ ]:
ds_ctsm

#### Take a quick look at the dataset.
- What are your coodinate variables?
- How long is the time dimensions?
- What variables do we have to look at?
- What are the long names of some of these variables? (HINT: try `ds_ctsm.TSOI`)
- What are other metadata are associated with this dataset? 

---

### Let's start plotting!

### Make a contour plot of soil moisture over time with depth on the y-axis

In [ ]:
# Plot H2OSOI
ds_ctsm.H2OSOI.plot(robust=True, y='levsoi')
# Invert the y-axis
plt.gca().invert_yaxis()
# Add the title
plt.title(neon_site)
# Show plot
plt.show()

### Plot vertical profiles of soil moisture 
This example is looking at one month of data over the top 2m of soil. Depth is on the y axis, and the plot is reversed so deeper soil levels are at the bottom.

In [ ]:
# Plot H20SOI at time 5; marker determines style of points
ds_ctsm.H2OSOI.isel(time=5).plot(y='levsoi',marker='o',ylim=(0,2))
# Invert y-axis
plt.gca().invert_yaxis()
# Add suptitle above plot title with time slice
plt.suptitle(neon_site)
# Show plot
plt.show()

It seems odd that surface soil layers are wetter than deeper ones.

### Plot a time series of soil moisture for a single soil level

In [ ]:
# Plot H2OSOI at soil level 4 with dot markers at each point
ds_ctsm.H2OSOI.isel(levsoi=4).plot(marker='o')
# Show plot
plt.show()

#### 

<div class="alert alert-block alert-success">
<b>Congratualtions:</b> 
    
You've now looked at some of the monthly output from CLM.

What other sites or variables would you like to look at?

Give it a shot, you can make lots of plots with all this data!
    
</div>
